In [34]:
import airgen
from airgen.utils.collect import data_collector
from typing import List, Tuple, Dict, Any, Optional, Callable

In [2]:
drones = [airgen.MultirotorClient()] *5

In [3]:
num_drones = 5
for i in range(num_drones):
    drone_name = f"Drone{i}" if i>0 else "Drone"
    drones[i].enableApiControl(True, vehicle_name=drone_name)
    drones[i].takeoffAsync(vehicle_name=drone_name)

In [6]:
drones[0].moveToPositionAsync(0, 10, -10, 5)

In [8]:
from grid.model.perception.detection.gdino import GroundingDINO

In [17]:
gdinomodel = GroundingDINO()
img = drones[0].getImages("front_center",[airgen.ImageType.Scene])[0][0]
boxes, phrases = gdinomodel.detect_object(img, "windmill .")
print(boxes, phrases)

[[ 49.85315   63.12695  101.855576 142.61145 ]
 [106.66642   80.96768  144.94434  137.97488 ]
 [185.32486  106.87101  195.88211  139.22798 ]
 [208.94939  106.346375 219.08446  141.56047 ]] ['windmill', 'windmill', 'windmill', 'windmill']


In [ ]:
from grid.model.perception.vlm.llava import LLaVA

In [19]:
llava = LLaVA()
outputs = llava.run(img, "What do you see?")
print(outputs)

 The image features a large, open field with a wind farm in the background. There are several wind turbines scattered throughout the field, with some closer to the foreground and others further away. The wind turbines are of varying sizes, and they are all connected to power lines, indicating that they are generating electricity.

In the foreground, there is a car parked, possibly belonging to someone visiting the wind farm. The overall scene gives a sense of the vastness of the field and the power of the wind turbines.


In [20]:
from grid.model.perception.depth.midas import MIDAS

In [24]:
midas = MIDAS()
depth = midas.run(img)
print(depth)

[[   0.          0.          0.       ...   49.099712   51.197495
    44.142357]
 [   0.          0.          0.       ...   47.814533   50.251526
    50.553577]
 [   0.          0.          0.       ...   48.603462   47.677307
    49.153477]
 ...
 [2542.075    2544.3564   2543.6848   ... 2498.5103   2491.9482
  2489.9546  ]
 [2559.432    2562.4492   2566.4429   ... 2520.8389   2509.9563
  2505.6707  ]
 [2566.123    2579.5022   2589.9307   ... 2555.2407   2523.0515
  2509.2966  ]]


In [30]:
num_episodes = 1
import numpy as np

x_bounds = (20, 50)  
y_bounds = (30, 50)  
num_values = 5     


x_values = np.random.uniform(x_bounds[0], x_bounds[1], num_values)
y_values = np.random.uniform(y_bounds[0], y_bounds[1], num_values)

for i in range(num_episodes):
    for i in range(num_drones):
        drone_name = f"Drone{i}" if i>0 else "Drone"
        drones[i].moveToPositionAsync(x_values[i], y_values[i], -10, 5, vehicle_name=drone_name)

In [63]:
def readSensors(client: airgen.MultirotorClient(), drone_name: 'str') -> dict:
    sensor_data = {}
    img = client.getImages("front_center",[airgen.ImageType.Scene], vehicle_name=drone_name)[0]
    sensor_data['rgb'] = img
    # sensor_data['depth'] = midas.run(img[0])
    # sensor_data['annot'] = gdinomodel.detect_object(img[0], "windmill .")
    return sensor_data

In [62]:
@data_collector(readSensors, time_delta=0.1)
def move_task(
    client: airgen.MultirotorClient, drone_name: 'str', position: Tuple[float], **kwargs
) -> None | Tuple[None, List[dict]]:
    client.moveToPositionAsync(position[0],position[1], position[2], position[3], vehicle_name=drone_name)

import numpy as np

x_bounds = (100, 50)  
y_bounds = (100, 50)  
num_values = 5     


x_values = np.random.uniform(x_bounds[0], x_bounds[1], num_values)
y_values = np.random.uniform(y_bounds[0], y_bounds[1], num_values)

for i in range(num_drones):
    drone_name = f"Drone{i}" if i>0 else "Drone"
    _, sensor_data = move_task(drones[i], drone_name, (x_values[i], y_values[i], -10, 5), _collect_data=True)
    
# for i, data in enumerate(sensor_data):
#     # lidar = data["lidar"]
#     # rgb, _ = data["rgb"]
#     # rr.log("grid/imagery",rr.Image(rgb))
#     # rr.log("pointcloud", rr.Points3D(np.array(lidar.point_cloud).reshape(-1, 3)))

print(f"collected {len(sensor_data)} measurements during moving task")

[AirGen][2024-11-08 01:19:43,451][WARNING]-[      collect.py:  69] - error in data collection task: readSensors() missing 1 required positional argument: 'drone_name', return data collection task:readSensors, is not complete


collected 0 measurements during moving task
